In [ ]:
# Training Methods

> Training, loss functions and metris will be developed here

In [ ]:
#| default_exp pytorch_training_and_loss

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
#import torch
#from torch.optim.lr_scheduler import CosineAnnealingLR
#from torch.optim.swa_utils import SWALR
#from torch import nn
#from torchvision.transforms.functional import to_pil_image


In [3]:
#| export
from pathlib import Path
from tqdm.auto import tqdm
from typing import List, Callable
import inspect
from statistics import mean
from cv_tools.core import *
from cv_tools.imports import *
import monai
from monai.metrics import DiceMetric

from fastcore.all import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.optim.lr_scheduler import CosineAnnealingLR

In [4]:
#| export

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        return F_loss


In [5]:
#| export
def iou_metric(preds, labels, threshold=0.5):
    # Convert predictions to binary format
    preds = (preds > threshold).float().squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    
    # Ensure labels are binary
    labels = labels.float().squeeze(1)  # Similarly squeeze if necessary

    # Compute intersection and union
    intersection = (preds * labels).sum((1, 2))  # Logical AND
    union = ((preds + labels) > 0).float().sum((1, 2))  # Logical OR

    # Calculate IoU
    iou = (intersection + 1e-6) / (union + 1e-6)  # Smoothing to avoid 0/0

    return iou.mean()


In [6]:
#| export
def false_positive_negative(outputs, labels):
    FP = ((outputs == 1) & (labels == 0)).float().sum()
    FN = ((outputs == 0) & (labels == 1)).float().sum()
    return FP, FN

In [ ]:
from segmentation_test.pytorch_model_development import *
from segmentation_test.dataloader_creation import *

In [ ]:

model = UNet(in_channels=1, out_channels=1,max_pool_only=True)

# dataloader

In [ ]:
im_path = Path(r'/home/hasan/workspace/data/microscopy_data/patch_images')
msk_path = Path(r'/home/hasan/workspace/data/microscopy_data/patch_masks')
im_path.ls()

(#1642) [Path('/home/hasan/workspace/data/microscopy_data/patch_images/img_147_p_7.png'),Path('/home/hasan/workspace/data/microscopy_data/patch_images/img_51_p_10.png'),Path('/home/hasan/workspace/data/microscopy_data/patch_images/img_164_p_1.png'),Path('/home/hasan/workspace/data/microscopy_data/patch_images/img_122_p_4.png'),Path('/home/hasan/workspace/data/microscopy_data/patch_images/img_134_p_1.png'),Path('/home/hasan/workspace/data/microscopy_data/patch_images/img_80_p_2.png'),Path('/home/hasan/workspace/data/microscopy_data/patch_images/img_77_p_8.png'),Path('/home/hasan/workspace/data/microscopy_data/patch_images/img_47_p_7.png'),Path('/home/hasan/workspace/data/microscopy_data/patch_images/img_78_p_10.png'),Path('/home/hasan/workspace/data/microscopy_data/patch_images/img_83_p_2.png')...]

In [ ]:
train_loader, val_loader = create_pytorch_dataloader(
    split_type='random',
    split_per=0.8,
    batch_size=2,
    image_path=im_path,
    mask_path=msk_path,
    transforms=None,
    num_workers=0

)

 Number of images found = 1642
 training dataset length = 1313 and validation dataset length=  329
torch.Size([1, 256, 256]) torch.Size([1, 256, 256])


In [ ]:
# Define model, optimizer, and loss
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
criterion = FocalLoss()
#| export
total_epochs = 10
# Define LR scheduler
scheduler = CosineAnnealingLR(optimizer, T_max=total_epochs, eta_min=0)
warmup_epochs = 0.1 * total_epochs
initial_lr = 0.002

In [ ]:
img, msk = next(iter(train_loader))

In [ ]:
img.shape, msk.shape

(torch.Size([2, 1, 256, 256]), torch.Size([2, 1, 256, 256]))

In [ ]:
focal_loss = FocalLoss()
# Training loop
for epoch in range(total_epochs):
    model.train()

    train_loss, train_iou_sum, train_fp_sum, train_fn_sum = 0.0, 0.0, 0.0, 0.0
    train_total = 0

    with tqdm(total=len(train_loader),desc=f'Epoch {epoch+1}/{total_epochs}', unit="batch") as pbar:

        for data, target in train_loader:
            optimizer.zero_grad()
            outputs = model(data)
            loss = focal_loss(outputs, target)
            loss.backward()
            optimizer.step()

            # Update metrics
            train_loss += loss.item()
            iou_score = iou_metric(outputs, target)
            train_iou_sum += iou_score.item()
            fp, fn = false_positive_negative(outputs, target)
            train_fp_sum += fp.item()
            train_fn_sum += fn.item()
            train_total += 1

            # Update progress bar
            pbar.set_postfix(
                              {'loss': loss.item(), 'iou': iou_score.item(), 'fp': fp.item(), 'fn': fn.item()}
                              )
            pbar.update()
    
    train_loss /= train_total
    train_iou = train_iou_sum / train_total
    train_fp = train_fp_sum / train_total
    train_fn = train_fn_sum / train_total

    # Validation
    model.eval()
    val_loss, val_iou_sum, val_fp_sum, val_fn_sum = 0.0, 0.0, 0.0, 0.0
    val_total = 0
    with torch.no_grad():
        for data, target in val_loader:
            outputs = model(data)
            loss = FocalLoss(outputs, target)

            # Update metrics
            val_loss += loss.item()
            iou_score = iou_metric(outputs, target)
            val_iou_sum += iou_score.item()
            fp, fn = false_positive_negative(outputs, target)
            val_fp_sum += fp.item()
            val_fn_sum += fn.item()
            val_total += 1

    val_loss /= val_total
    val_iou = val_iou_sum / val_total
    val_fp = val_fp_sum / val_total
    val_fn = val_fn_sum / val_total

    # Adjust learning rate after warm-up
    if epoch < warmup_epochs:
        lr = initial_lr * (epoch / warmup_epochs)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    else:
        scheduler.step()

    # Save model with metrics in the filename
    torch.save(model.state_dict(), f'model_epoch_{epoch+1}_valiou_{val_iou:.2f}_fp_{val_fp:.0f}_fn_{val_fn:.0f}.pth')

    # Print epoch summary
    print(f'Epoch {epoch+1}/{total_epochs} - Train Loss: {train_loss:.4f}, IoU: {train_iou:.4f}, FP: {train_fp}, FN: {train_fn}')
    print(f'Validation Loss: {val_loss:.4f}, IoU: {val_iou:.4f}, FP: {val_fp}, FN: {val_fn}')


Epoch 1/10:   0%|          | 0/657 [00:00<?, ?batch/s]

In [ ]:
#| export
def validate_front(
        model:nn.Module, 
        dataloader:DataLoader,
        loss_fn:monai.losses.dice, 
        metrics:List[Callable]=[],
        device:str='cpu',
        threshold:float=0.5
        ):

    model.eval()
    model.to(device)
    val_losses = []

    epoch_metrics = {metric.__name__ if inspect.isfunction(metric) else metric.__class__.__name__: [] for metric in metrics}
    with torch.no_grad():
        for batch in dataloader:
            img, msk = batch
            img = img.to(device)
            msk = msk.to(device)
            ground_truth_mask = msk
            outputs = model(img)
            #predicted_masks = outputs.pred_masks.squeeze(1)
            #ground_truth_masks = batch["ground_truth_mask"].float().to(device)
            #loss = loss_fn(outputs.squeeze(1), ground_truth_mask.stqueeze(1))
            loss = loss_fn(outputs, ground_truth_mask)
            val_losses.append(loss.item())
            for metric in metrics:
                metric_value = metric(
                    outputs>=threshold, 
                    ground_truth_mask>=threshold
                    )
                if inspect.isfunction(metric):
                    epoch_metrics[metric.__name__].append(
                        metric_value)
                else:
                    epoch_metrics[metric.__class__.__name__].append(
                        metric_value)
        
        for metric_name, values in epoch_metrics.items():
            print(f'Mean Validation {metric_name}: {mean(values)}')


    model.train()

    return mean(val_losses), np.mean(list(epoch_metrics.values()))

In [ ]:
#| export
def train_and_validate_front(
        model: nn.Module,  # Model to train
        num_epochs: int,  # Number of epochs to train for
        optimizer: torch.optim.Optimizer, # Optimizer to use
        scheduler: torch.optim.lr_scheduler._LRScheduler,  # Learning rate scheduler
        train_dataloader: DataLoader,  # DataLoader for training data
        val_dataloader: DataLoader, # DataLoader for validation data
        loss_fn: nn.Module,  # Loss function used for training
        metrics: List[Callable],  # List of metrics to compute
        threshold: float = 0.5,  # Threshold for binarizing the predictions
        device: str = 'cuda' if torch.cuda.is_available() else 'cpu', # Device to train on
        save_path: str = './models', # Path to save the model after training
        model_fn: str = 'model', # Base filename of model
        ):
    """Train and validate a model with the given parameters."""
    
    model = model.float().to(device)
    best_val_metric = float('-inf')
    best_epoch = -1

    for epoch in range(num_epochs):
        model.train()
        epoch_losses = []
        epoch_metrics = {metric.__name__ if callable(metric) else type(metric).__name__: [] for metric in metrics}
        print(f'###################')
        print(f'Started epoch = {epoch + 1}')

        progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs}')

        for batch in progress_bar:
            img, msk = batch
            img, msk = img.float().to(device), msk.to(device)
            
            optimizer.zero_grad()
            outputs = model(img)
            loss = loss_fn(outputs, msk)
            
            loss.backward()
            optimizer.step()
            
            epoch_losses.append(loss.item())
            
            with torch.no_grad():
                predicted_masks = (outputs > threshold).float()
                for metric in metrics:
                    metric_value = metric(predicted_masks, msk)
                    metric_name = metric.__name__ if callable(metric) else type(metric).__name__
                    epoch_metrics[metric_name].append(metric_value.item() if isinstance(metric_value, torch.Tensor) else metric_value)
            
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        if scheduler:
            scheduler.step()

        # Validation phase
        model.eval()
        val_loss, val_metrics = validate_front(
            model=model, 
            dataloader=val_dataloader, 
            loss_fn=loss_fn, 
            metrics=metrics,
            threshold=threshold,
            device=device)

        # Print epoch results
        print(f'Epoch: {epoch + 1}')
        print(f'Mean Training Loss: {np.mean(epoch_losses):.4f}')
        print(f'Validation Loss: {val_loss:.4f}')
        for metric_name, values in epoch_metrics.items():
            train_metric = np.mean(values)
            val_metric = val_metrics[metric_name]
            print(f'Train {metric_name}: {train_metric:.4f}, Validation {metric_name}: {val_metric:.4f}')

        # Check if the current validation metric is better than the best so far
        current_val_metric = val_metrics[metrics[0].__name__ if callable(metrics[0]) else type(metrics[0]).__name__]
        if current_val_metric > best_val_metric:
            best_val_metric = current_val_metric
            best_epoch = epoch + 1
            
            # Save the model if it's the best so far
            save_dir = Path(save_path)
            save_dir.mkdir(parents=True, exist_ok=True)
            model_save_path = save_dir / f"{model_fn}_best_val_{best_val_metric:.4f}_epoch_{best_epoch}.pth"
            torch.save({
                'epoch': best_epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_metric': best_val_metric,
            }, model_save_path)
            print(f'New best model saved to {model_save_path}')

        print(f'###################')
        print(f'Ended epoch = {epoch + 1}')

    print(f'Training completed. Best validation metric: {best_val_metric:.4f} at epoch {best_epoch}')


In [ ]:
#| export
def train_segmentation_model(
                            train_loader,
                            val_loader,
                            model,
                            optimizer,
                            scheduler, 
                            loss_fn, 
                            model_save_path, 
                            warmup_epochs, 
                            total_epochs, 
                            initial_lr, 
                            dtype=torch.float,
                            device='cuda'):
    # Set the device for training
    device = torch.device(device if torch.cuda.is_available() else 'cpu')
    model = model.to(device=device, dtype=dtype)

    for epoch in range(total_epochs):
        model.train()

        train_loss, train_iou_sum, train_fp_sum, train_fn_sum = 0.0, 0.0, 0.0, 0.0
        train_total = 0

        with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{total_epochs}', unit="batch") as pbar:
            for data, target in train_loader:
                # Move data and target to the specified device and data type
                data = data.to(device=device, dtype=dtype)
                target = target.to(device=device, dtype=dtype)

                optimizer.zero_grad()
                outputs = model(data)
                loss = loss_fn(outputs, target)
                loss.backward()
                optimizer.step()

                # Update metrics
                train_loss += loss.item()
                iou_score = iou_metric(outputs, target)
                train_iou_sum += iou_score.item()
                fp, fn = false_positive_negative(outputs, target)
                train_fp_sum += fp.item()
                train_fn_sum += fn.item()
                train_total += 1

                # Update progress bar
                pbar.set_postfix({'loss': loss.item(), 'iou': iou_score.item(), 'fp': fp.item(), 'fn': fn.item()})
                pbar.update()

        # ... Rest of the training loop as before ...
        train_loss /= train_total
        train_iou = train_iou_sum / train_total
        train_fp = train_fp_sum / train_total
        train_fn = train_fn_sum / train_total

        # Validation
        model.eval()
        val_loss, val_iou_sum, val_fp_sum, val_fn_sum = 0.0, 0.0, 0.0, 0.0
        val_total = 0
        with torch.no_grad():
            for data, target in val_loader:

                data = data.to(device=device, dtype=dtype)
                target = target.to(device=device, dtype=dtype)

                outputs = model(data)
                loss = loss_fn(outputs, target)

                # Update metrics
                val_loss += loss.item()
                iou_score = iou_metric(outputs, target)
                val_iou_sum += iou_score.item()
                fp, fn = false_positive_negative(outputs, target)
                val_fp_sum += fp.item()
                val_fn_sum += fn.item()
                val_total += 1

        val_loss /= val_total
        val_iou = val_iou_sum / val_total
        val_fp = val_fp_sum / val_total
        val_fn = val_fn_sum / val_total

        # Adjust learning rate after warm-up
        if epoch < warmup_epochs:
            lr = initial_lr * (epoch / warmup_epochs)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
        else:
            scheduler.step()

          # Save model
        torch.save(model.state_dict(), f'{model_save_path}/model_epoch_{epoch+1}_valiou_{val_iou:.2f}_fp_{val_fp:.0f}_fn_{val_fn:.0f}.pth')


        # Print epoch summary
        print(f'Epoch {epoch+1}/{total_epochs} - Train Loss: {train_loss:.4f}, IoU: {train_iou:.4f}, FP: {train_fp}, FN: {train_fn}')
        print(f'Validation Loss: {val_loss:.4f}, IoU: {val_iou:.4f}, FP: {val_fp}, FN: {val_fn}')



In [7]:
#| hide
import nbdev; nbdev.nbdev_export('10_training_pytorch.ipynb')